In [52]:
import laspy
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import open3d as o3d


# Load LAS file
las = laspy.read("/Users/hdong/Projects/RoofSegmentation/outputs/P5123C2_9_g_h_b2_n_c6.las")
points = np.vstack((las.x, las.y, las.z)).transpose()

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)

# # Normalize the point cloud
# # scaler = StandardScaler()
# # points_normalized = scaler.fit_transform(points)

# pcd_downsampled = pcd.voxel_down_sample(voxel_size=0.5)  # Adjust voxel size as needed
# points = np.asarray(pcd_downsampled.points)

In [53]:
from sklearn.linear_model import RANSACRegressor

def ransac_plane_segmentation(points, min_samples=3, residual_threshold=0.1, max_trials=10000):
    best_inliers = []
    remaining_points = points.copy()
    
    while len(remaining_points) > min_samples:
        # Fit plane using RANSAC
        ransac = RANSACRegressor(min_samples=min_samples, residual_threshold=residual_threshold, max_trials=max_trials)
        ransac.fit(remaining_points[:, :2], remaining_points[:, 2])
        
        # Get inliers
        inlier_mask = ransac.inlier_mask_
        inliers = remaining_points[inlier_mask]
        
        # Check if we found a significant plane
        if len(inliers) > min_samples:
            best_inliers.append(inliers)
            remaining_points = remaining_points[~inlier_mask]
        else:
            break
    
    return best_inliers, remaining_points

# Segment planes
segmented_planes, outliers = ransac_plane_segmentation(points)


In [46]:
from sklearn.cluster import DBSCAN

def refine_and_classify_planes(segmented_planes, eps=0.5, min_samples=10):
    refined_planes = []
    for plane in segmented_planes:
        # Use DBSCAN to cluster points within each plane
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(plane)
        
        # Separate clusters
        unique_labels = np.unique(labels)
        for label in unique_labels:
            if label != -1:  # Ignore noise points
                refined_planes.append(plane[labels == label])
    
    return refined_planes

# Refine and classify planes
roof_planes = refine_and_classify_planes(segmented_planes)


In [56]:
import open3d as o3d

def visualize_segmented_planes(planes):
    # colors = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1]]
    pcd = o3d.geometry.PointCloud()
    
    for i, plane in enumerate(planes):
        plane_pcd = o3d.geometry.PointCloud()
        plane_pcd.points = o3d.utility.Vector3dVector(plane)
        # plane_pcd.paint_uniform_color(colors[i % len(colors)])
        plane_pcd.paint_uniform_color(np.random.rand(3))
        pcd += plane_pcd
    
    o3d.visualization.draw_geometries([pcd])

# Visualize segmented roof planes
visualize_segmented_planes(segmented_planes)


: 

In [6]:
def save_segmented_planes_with_labels_to_las(planes, output_file="segmented_planes_with_labels.las"):
    # Combine all planes into a single array and assign labels
    all_points = []
    labels = []
    for label, plane in enumerate(planes):
        all_points.append(plane)
        labels.extend([label] * len(plane))

    all_points = np.vstack(all_points)
    labels = np.array(labels)

    # Create a new LAS file
    header = laspy.LasHeader(point_format=3, version="1.2")
    las = laspy.LasData(header)

    # Assign x, y, z coordinates
    las.x = all_points[:, 0]
    las.y = all_points[:, 1]
    las.z = all_points[:, 2]

    # Add labels as a new dimension (classification field)
    las.classification = labels

    # Save to file
    las.write(output_file)
    print(f"Segmented planes with labels saved to {output_file}")


In [55]:
save_segmented_planes_with_labels_to_las(segmented_planes, "/Users/hdong/Projects/RoofSegmentation/outputs/P5123C2_9_b_2.classified_roofs_2.las")

Segmented planes with labels saved to /Users/hdong/Projects/RoofSegmentation/outputs/P5123C2_9_b_2.classified_roofs_2.las


In [ ]:
# Assign labels to the LAS file points
labels = np.zeros(len(points))  # Initialize labels as 0
current_label = 1

for inliers in plane_inliers:
    indices = np.asarray(inliers)
    labels[indices] = current_label
    current_label += 1

# Save the LAS file with labels

las.add_extra_dim(laspy.ExtraBytesParams(
    name="classification",
    type=np.uint8
))
# las.add_extra_dim(name="classification", data_type=laspy.ExtraBytesParams.DATA_TYPE_UINT8)
las.classification = labels.astype(np.uint8)

las.write("/Users/hdong/Projects/RoofSegmentation/outputs/P5123C2_9_b_2.classified_roofs.las")